# Machine Learning Foundation

## Course 5, Part g: RNN DEMO

## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).

In [1]:
from tensorflow import keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.datasets import imdb
from tensorflow.keras import initializers
# import keras
# from keras.preprocessing import sequence
# from keras.models import Sequential
# from keras.layers import Dense, Embedding
# from keras.layers import SimpleRNN
# from keras.datasets import imdb
# from keras import initializers

In [2]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [3]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
C:\Users\MOHAMAD\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


25000 train sequences
25000 test sequences


C:\Users\MOHAMAD\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [7]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [8]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99])

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [10]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [11]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 50)          1000000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 5)                 280       
_________________________________________________________________
dense (Dense)                (None, 1)                 6         
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [12]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [13]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 17s 22ms/step - loss: 0.6580 - accuracy: 0.6274 - val_loss: 0.5977 - val_accuracy: 0.6858
Epoch 2/10
782/782 [==============================] - 16s 21ms/step - loss: 0.5536 - accuracy: 0.7178 - val_loss: 0.5418 - val_accuracy: 0.7248
Epoch 3/10
782/782 [==============================] - 16s 20ms/step - loss: 0.4960 - accuracy: 0.7599 - val_loss: 0.5118 - val_accuracy: 0.7429
Epoch 4/10
782/782 [==============================] - 16s 21ms/step - loss: 0.4564 - accuracy: 0.7868 - val_loss: 0.4899 - val_accuracy: 0.7558
Epoch 5/10
782/782 [==============================] - 16s 20ms/step - loss: 0.4278 - accuracy: 0.8036 - val_loss: 0.4732 - val_accuracy: 0.7670
Epoch 6/10
782/782 [==============================] - 16s 21ms/step - loss: 0.4066 - accuracy: 0.8169 - val_loss: 0.4633 - val_accuracy: 0.7742
Epoch 7/10
782/782 [==============================] - 16s 21ms/step - loss: 0.3908 - accuracy: 0.8246 - val_loss: 0.4559 - val_accuracy:

In [14]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 2s 3ms/step - loss: 0.4543 - accuracy: 0.7849
Test score: 0.45425984263420105
Test accuracy: 0.7848799824714661


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?

In [15]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
C:\Users\MOHAMAD\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\MOHAMAD\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different length

In [16]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [17]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [18]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 39s 50ms/step - loss: 0.6248 - accuracy: 0.6499 - val_loss: 0.5503 - val_accuracy: 0.7200
Epoch 2/10
782/782 [==============================] - 38s 49ms/step - loss: 0.4896 - accuracy: 0.7660 - val_loss: 0.4946 - val_accuracy: 0.7670
Epoch 3/10
782/782 [==============================] - 38s 49ms/step - loss: 0.4263 - accuracy: 0.8080 - val_loss: 0.4557 - val_accuracy: 0.7828
Epoch 4/10
782/782 [==============================] - 39s 49ms/step - loss: 0.3803 - accuracy: 0.8353 - val_loss: 0.4440 - val_accuracy: 0.7856
Epoch 5/10
782/782 [==============================] - 38s 49ms/step - loss: 0.3475 - accuracy: 0.8512 - val_loss: 0.4090 - val_accuracy: 0.8157
Epoch 6/10
782/782 [==============================] - 39s 50ms/step - loss: 0.3258 - accuracy: 0.8636 - val_loss: 0.4023 - val_accuracy: 0.8184
Epoch 7/10
782/782 [==============================] - 37s 47ms/step - loss: 0.3088 - accuracy: 0.8712 - val_loss: 0.4014 - val_accuracy:

In [19]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [20]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [21]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [22]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 38s 48ms/step - loss: 0.6796 - accuracy: 0.5389 - val_loss: 0.6428 - val_accuracy: 0.6449
Epoch 2/10
782/782 [==============================] - 36s 46ms/step - loss: 0.6115 - accuracy: 0.7168 - val_loss: 0.6019 - val_accuracy: 0.7067
Epoch 3/10
782/782 [==============================] - 36s 46ms/step - loss: 0.5735 - accuracy: 0.7693 - val_loss: 0.5917 - val_accuracy: 0.7210
Epoch 4/10
782/782 [==============================] - 37s 47ms/step - loss: 0.5491 - accuracy: 0.7938 - val_loss: 0.5623 - val_accuracy: 0.7880
Epoch 5/10
782/782 [==============================] - 37s 47ms/step - loss: 0.5317 - accuracy: 0.8131 - val_loss: 0.5481 - val_accuracy: 0.8022
Epoch 6/10
782/782 [==============================] - 38s 49ms/step - loss: 0.5167 - accuracy: 0.8211 - val_loss: 0.5378 - val_accuracy: 0.8056
Epoch 7/10
782/782 [==============================] - 38s 48ms/step - loss: 0.5026 - accuracy: 0.8287 - val_loss: 0.5294 - val_accuracy:

In [23]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 37s 47ms/step - loss: 0.4574 - accuracy: 0.8383 - val_loss: 0.4958 - val_accuracy: 0.8095
Epoch 2/10
782/782 [==============================] - 36s 46ms/step - loss: 0.4469 - accuracy: 0.8415 - val_loss: 0.4887 - val_accuracy: 0.8088
Epoch 3/10
782/782 [==============================] - 37s 47ms/step - loss: 0.4160 - accuracy: 0.8405 - val_loss: 0.4455 - val_accuracy: 0.8022
Epoch 4/10
782/782 [==============================] - 38s 49ms/step - loss: 0.3757 - accuracy: 0.8369 - val_loss: 0.4613 - val_accuracy: 0.7725
Epoch 5/10
782/782 [==============================] - 38s 48ms/step - loss: 0.3638 - accuracy: 0.8449 - val_loss: 0.4603 - val_accuracy: 0.7727
Epoch 6/10
782/782 [==============================] - 37s 47ms/step - loss: 0.3555 - accuracy: 0.8476 - val_loss: 0.4231 - val_accuracy: 0.8130
Epoch 7/10
782/782 [==============================] - 37s 47ms/step - loss: 0.3470 - accuracy: 0.8507 - val_loss: 0.4471 - val_accuracy:

---
### Machine Learning Foundation (C) 2020 IBM Corporation